# 세션 20 — Transformer (DistilBERT) 감정분석 — GLUE/SST-2 — 2025 버전

In [ ]:
# torchvision과 torchaudio를 추가하여 torch 버전과 호환되도록 함께 업데이트합니다.
!pip -q install -U "torch>=2.2,<3.0" "torchvision" "torchaudio" "datasets>=3.0.1" "transformers>=4.45.2" "accelerate>=1.0.1" "evaluate>=0.4.2"

import torch, transformers, datasets, evaluate
import numpy as np

print("PyTorch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("Transformers:", transformers.__version__, "| Datasets:", datasets.__version__)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
from datasets import load_dataset
ds = load_dataset("glue", "sst2")
print(ds)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

MODEL="distilbert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(MODEL,use_fast=True)
def preprocess(ex): return tokenizer(ex["sentence"], truncation=True, max_length=256)
enc=ds.map(preprocess, batched=True, remove_columns=["sentence","idx"])
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
model=AutoModelForSequenceClassification.from_pretrained(MODEL,num_labels=2).to(device)

In [ ]:
import evaluate
from transformers import TrainingArguments, Trainer
import torch

# 지표 로드
acc = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def metrics(p):
    predictions, labels = p
    # predictions가 튜플로 나오는 경우(logits 외 다른 요소가 있는 경우)를 대비해 분기 처리하거나
    # 일반적인 경우 predictions 자체가 logits입니다.
    # 여기서는 numpy 배열이라고 가정하고 처리합니다.
    preds = predictions.argmax(-1)

    return {
        "acc": acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="binary")["f1"]
    }

# TrainingArguments 설정
# >> 학습 스케쥴(프로젝트 한다면 WBS(Work Breakdown Sheet))
args = TrainingArguments(
    output_dir="/content/sst2_2025",
    eval_strategy="epoch",          # 수정됨: evaluation_strategy -> eval_strategy
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    # 학습이 끝난 가장 좋은 모델 자동 로드
    fp16=torch.cuda.is_available(), # GPU가 있을 때만 fp16 사용
    report_to="none",
    seed=2025
)
# fp16 : float precision(부동소수점 정밀도):
# >> GPU가 있을 때만 fp16 사용 (16bit) 메모리 절약 >> 속도 향상

# Trainer 초기화
# 학습 스케쥴대로 자동화
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=enc["train"],
    eval_dataset=enc["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=metrics
)
# data_collator : 정렬 도구 (문장 길이 맞추기)

In [ ]:
txt=["This movie was amazing!","Worst film ever."]
inp=tokenizer(txt,return_tensors="pt",padding=True,truncation=True,max_length=256).to(model.device)

with torch.no_grad(): out=torch.softmax(model(**inp).logits,dim=-1).cpu().numpy()
# **inp : input(입력된 인자 argument) 언패킹해서 전달
# >>(dictionary 형태로 되어 있는 것을 키워드 인수로 풀어서 전달)
# inp = {"id": [].....} >> model(input_id = .....)

for t,p in zip(txt,out): print(f"{t}\n→ Negative={p[0]:.3f}, Positive={p[1]:.3f}")